In [2]:
import pandas as pd
import numpy as np
import datetime as dt

In [3]:
# Retrieve the signals, allocations and leverage from the saved files
df_sig = pd.read_pickle("China Daily Signals.pickle")
df_lev = pd.read_pickle("China Daily Leverage.pickle")

In [5]:
# Create a dataframe with the signal variable
# Shift signal down one so that each day has the previous days signal
df = pd.DataFrame(data=df_sig["Signal"].shift(+1), index=df_sig.index)

In [6]:
# Set the start date and end date
startdate = dt.date(2011, 9, 1)
enddate = dt.date(2021, 9, 30)

# Filter for the required period
df = df[startdate : enddate]

In [7]:
# Join the cash dataframe to df
df = df.merge(df_sig[[ 'FXI']], how='left', on='Date')

In [8]:
# Calculate the return for the index and add as a new variable
df["FXI_Return"] = (df["FXI"] / df["FXI"].shift(+1)) - 1.0

In [9]:
# Include the leverage
df = df.merge(df_lev[['Leverage']], how='left', on='Date')

In [11]:
# Shift the leverage down one so that each day has the previous days leverage
df["Leverage"]=df["Leverage"].shift(+1)

In [12]:
# Calculate the return for each day
df["Return"] = df["FXI_Return"]

In [13]:
# Convert the signal to long/short position
df["Position"] = (df["Signal"]*2.0) - 1.0

In [ ]:
# Apply the position
df["Return"] = df["Return"]*df["Position"]

In [14]:
# Apply the leverage
df["Return"] = df["Return"]*df["Leverage"]

In [15]:
# Set the first return to zero to get rid of the #NaN
df["Return"].iloc[0] = 0.0

In [16]:
# Initialise the capital variable
df["Capital"] = 100.0

In [17]:
# Loop through to calculate the capital each day
for i in range(1, df.shape[0]):
    df["Capital"][i] = df["Capital"][i-1]*(1.0 + df["Return"][i])

In [18]:
# Save the data to excel file
df.to_excel("China Returns and Capital.xlsx")

# Also save the data to pickle file
df.to_pickle("China Returns and Capital.pickle")

In [20]:
df.tail()

,Signal,FXI,FXI_Return,Leverage,Return,Position,Capital
Date,,,,,,,
2021-09-24,1.0,38.24,-0.020993,2.0,-0.041987,1.0,93.392242
2021-09-27,1.0,38.79,0.014383,2.0,0.028766,1.0,96.078734
2021-09-28,1.0,38.84,0.001289,2.0,0.002578,1.0,96.326423
2021-09-29,1.0,38.56,-0.007209,2.0,-0.014418,1.0,94.937577
2021-09-30,1.0,38.93,0.009595,2.0,0.019191,1.0,96.759512


In [21]:
years = (enddate - startdate).days/365.25

In [22]:
years

10.080766598220396

In [23]:
total_return = df.loc[[enddate]]["Capital"]/100.0

In [24]:
total_return

Date
2021-09-30    0.967595
Name: Capital, dtype: float64

In [25]:
cagr = total_return**(1/years)-1.0

In [26]:
cagr

Date
2021-09-30   -0.003262
Name: Capital, dtype: float64

In [14]:
2+2

4

In [230]:
df.to_excel("Returns and Capital.xlsx")